In [3]:
"""
R001_Fall_2023_AI_Reproducibility__
|
R001_Creating_the_RS_superset.ipynb
Created on Tue Sep  5 20:25:12 2023
@author: Rochana Obadage
"""

# ``RS documents download and metadata extraction - Superset`` 

### `dependencies`
> <font face="consolas" color='#000080'><b>1. python:</b><br> 
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;3.8.17<br><br>
<b>2. Jupyter Notebook:</b><br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;notebook server: 6.5.4<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Kernel Information: Python 3.8.17 (default, Jul  5 2023, 20:44:21) [MSC v.1916 64 bit (AMD64)]<br><br>
<b>3. conda installations:</b><br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;pandas==2.0.3<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Beatifulsoup4==4.11.1<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Selenium==4.11.2<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;conda install -c conda-forge selenium<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;webdriver-manager=4.0.0<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;conda install -c conda-forge webdriver-manager<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;bibtexparser-1.4.0<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;conda install -c conda-forge bibtexparser!<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;pdfminer.six-20221105<br>
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;conda install -c conda-forge pdfminer.six<br>

</font>


In [1]:
from bs4 import BeautifulSoup, NavigableString, Tag
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.action_chains import ActionChains

from bibtexparser.bparser import BibTexParser
from bibtexparser.customization import convert_to_unicode

import pandas as pd
import requests
import json
import urllib
import time
import string
import os
import shutil
import math
import bibtexparser

## ``All RS documents from 2021 to back ``

In [34]:
FOLDER_NAME = 'RS_2019_2021_REP_ALL/'

if not os.path.exists(FOLDER_NAME):
    os.makedirs(FOLDER_NAME)

# instantiate options 
options = webdriver.ChromeOptions() 
 
# run browser in headless mode 
options.headless = True 
 
# instantiate driver 
driver = webdriver.Chrome(service=ChromeService(
    ChromeDriverManager().install()), options=options) 

parser = BibTexParser()
parser.customization = convert_to_unicode

# load website 
url = 'https://rescience.github.io/read'

# get the entire website content 
driver.get(url) 

aop_items = []
printable = set(string.printable)
soup = BeautifulSoup(driver.page_source, 'lxml')
print(type(soup))

results_MLRC_2021 = soup.find_all("ol", class_="bibliography")[1]
results_MLRC_2020 = soup.find_all("ol", class_="bibliography")[3]
results_NeurIPS_2019 = soup.find_all("ol", class_="bibliography")[6]
results_ICLR_2019 = soup.find_all("ol", class_="bibliography")[9]

results_for_all_years = {'2021_MLRC':results_MLRC_2021,'2020_MLRC':results_MLRC_2020,
                         '2019_NeurIPS':results_NeurIPS_2019,'2019_ICLR':results_ICLR_2019}

challenge_year = '2021'
start_key = 46
internal_key = 0

# meta_data
key_for_all_RS = ""
title_rs = ""
authors_rs = ""
article_type = ""

keywords = ""
abstract = ""
pdf_url_rs = ""
code_url_rs = ""

org_paper_url = ""
doi_rs = ""
code_doi_rs = ""
doi_org = ""
venue_name_rs = ""
challenge_year_rs = ""
review_url_rs = ""

journal_rs = ""
volume_journal_rs = ""
issue_journal_rs = "" 
published_year_journal_rs = ""

domain = ""
entry_type = ""

all_meta_data_keys = ['key_for_all_RS', 'title_rs', 'authors_rs', 'article_type', 'keywords', 'abstract', 
                      'pdf_url_rs', 'code_url_rs', 'org_paper_url', 'doi_rs', 'code_doi_rs', 'doi_org', 'venue_name_rs', 
                      'challenge_year_rs', 'review_url_rs', 'journal_rs', 'volume_journal_rs', 'issue_journal_rs', 
                      'published_year_journal_rs', 'domain', 'entry_type']

row_items = []

for year, result_set in results_for_all_years.items():
    challenge_year, challenge = year.split('_')
    
    for key, item in enumerate(result_set):
        if isinstance(item, NavigableString):
            continue

        if isinstance(item, Tag):
            internal_key +=1

            bib = item.select("div pre")[0]
            library_ = bibtexparser.loads(bib.text)
            
            key_for_all_rs = 'RS_{:0>3}_{}_{}_{:0>2}'.format(start_key,challenge,challenge_year,internal_key)
            print(key_for_all_rs)
            start_key += 1
    
            if library_.entries == []:
                title_rs = bib.text
                print(bib.text)
    
            else:
                details_dict = library_.entries[0]
                
                key_for_all_RS = key_for_all_rs
                title_rs = details_dict['title'].replace('{','').replace('}','')
                print(title_rs)
                authors_rs = details_dict['author']
                article_type = details_dict['type']

                keywords = details_dict['keywords']
                abstract = 'DO EXTRACT FROM review_url'
                pdf_url_rs = details_dict['url']
                code_url_rs = details_dict['code_url']

                org_paper_url = 'DO EXTRACT FROM review_url'
                doi_rs = details_dict['doi']
                code_doi_rs = details_dict['code_doi']
                doi_org = ''
                venue_name_rs = challenge
                challenge_year_rs = challenge_year
                review_url_rs = details_dict['review_url']

                journal_rs = details_dict['journal']
                volume_journal_rs = details_dict['volume']
                issue_journal_rs = details_dict['number'] 
                published_year_journal_rs = details_dict['year']

                domain = details_dict['domain']
                entry_type = details_dict['ENTRYTYPE']
                
                if start_key > 128:

                    response = requests.get(pdf_url_rs)
                    file_save_name = FOLDER_NAME+key_for_all_RS +'.pdf'
                    pdf = open(file_save_name, 'wb')
                    pdf.write(response.content)
                    pdf.close()
            
            dict_ = dict()
            
            for key in all_meta_data_keys:
                dict_.update({key:eval(key)})

            row_items.append(dict_)
            
    internal_key = 0
    
driver.quit()
    
df = pd.DataFrame(row_items)
df.to_csv("RS_ALL_IN_ONE_2019_2021.csv", index = False)

C:\Users\cs_oruma001\AppData\Local\Temp\ipykernel_22112\1212157067.py:11: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


<class 'bs4.BeautifulSoup'>
RS_046_MLRC_2021_01
ML Reproducibility Challenge 2021
RS_047_MLRC_2021_02
[Re] Counterfactual Generative Networks
RS_048_MLRC_2021_03
[Re] Does Self-Supervision Always Improve Few-Shot Learning?
RS_049_MLRC_2021_04
[Re] Weakly-Supervised Semantic Segmentation via Transformer Explainability
RS_050_MLRC_2021_05
[Re] Reproducibility Study of “Counterfactual Generative Networks”
RS_051_MLRC_2021_06
[Re] Data-Driven Methods for Balancing Fairness and Efficiency in Ride-Pooling
RS_052_MLRC_2021_07
[¬Re] Hate Speech Detection based on Sentiment Knowledge Sharing
RS_053_MLRC_2021_08
[Re] Reproducibility Study - SCOUTER: Slot Attention-based Classifier for Explainable Image Recognition
RS_054_MLRC_2021_09
[Re] AdaBelief Optimizer: Adapting Stepsizes by the Belief in Observed Gradients
RS_055_MLRC_2021_10
[Re] GANSpace: Discovering Interpretable GAN Controls
RS_056_MLRC_2021_11
[Re] Replication study of "Privacy-preserving Collaborative Learning"
RS_057_MLRC_2021_12
[

RS_129_ICLR_2019_01
ICLR Reproducibility Challenge 2019
RS_130_ICLR_2019_02
[Re] Meta-learning with differentiable closed-form solvers
RS_131_ICLR_2019_03
[Re] Variational Sparse Coding
RS_132_ICLR_2019_04
[Re] Learning Neural PDE Solvers with Convergence Guarantees ICLR Reproducibility Challenge 2019
RS_133_ICLR_2019_05
[Re] h-detach: Modifying the LSTM gradient towards better optimization


In [ ]:
[{'keywords': 'multi-type inter-novel-protein interaction, graph neural networks, fair evaluation, python, rescience c, replication, machine learning', 
  'domain': 'ML Reproducibility Challenge 2021', 'language': 'Python', 
  'type': 'Replication', 'review_url': 'https://openreview.net/forum?id=Hc8GOhfmhRF', 
  'data_doi': '', 'data_url': '', 'code_swh': 'swh:1:dir:6eedc394f714587f35840bee0aac3e675bfa6c5a', 
  'code_doi': '10.5281/zenodo.6511807', 'code_url': 'https://github.com/zrimseku/Reproducibility-Challenge', 
  'url': 'https://zenodo.org/record/6574721/files/article.pdf', 'doi': '10.5281/zenodo.6574721', 'pages': '{#47}', 
  'number': '2', 'volume': '8', 'month': 'May', 'year': '2022', 'journal': 'ReScience C', 
  'title': '{[Re] Learning Unknown from Correlations: Graph Neural Network for Inter-novel-protein Interaction Prediction}', 
  'author': 'Zrimšek, Urša', 'ENTRYTYPE': 'article', 'ID': 'Zrimsek:2022'}]

## ``All RS documents from MLRC 2022 ``

In [33]:
df_accept_1 = pd.read_csv('MLRC_2022_Accepted_ALL_in_one.csv')

challenge_year = '2022'
start_key = 1
internal_key = 0

# meta_data
key_for_all_RS = ""
title_rs = ""
authors_rs = ""
article_type = ""

keywords = ""
abstract = ""
pdf_url_rs = ""
code_url_rs = ""

org_paper_url = ""
doi_rs = ""
code_doi_rs = ""
doi_org = ""

venue_name_rs = ""
challenge_year_rs = ""

review_url_rs = ""

journal_rs = ""
volume_journal_rs = ""
issue_journal_rs = "" 
published_year_journal_rs = ""


domain = ""
entry_type = ""

all_meta_data_keys = ['key_for_all_RS', 'title_rs', 'authors_rs', 'article_type', 'keywords', 'abstract', 
                      'pdf_url_rs', 'code_url_rs', 'org_paper_url', 'doi_rs', 'code_doi_rs', 'doi_org', 'venue_name_rs', 
                      'challenge_year_rs', 'review_url_rs', 'journal_rs', 'volume_journal_rs', 'issue_journal_rs', 
                      'published_year_journal_rs', 'domain', 'entry_type']

row_items = []

for index, row in df_accept_1.iterrows():

    # Page data extraction
    page = requests.get(row["re_details"])
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="content")
    
    # Meta-Data extraction
    details = soup.find(id="__NEXT_DATA__")
    full_dict = json.loads(details.text)
    content_dict = full_dict['props']['pageProps']['forumNote']['content']
    
    print(index+1)
    key_for_all_rs = 'RS_{:0>3}_{}_{}_{:0>2}'.format(index+1,challenge,challenge_year,index+1)

    keys_initial_list = ['title','authors','TL;DR','keywords','abstract','pdf',
                        'paper_url','paper_venue','venue_name','paperhash',
                         'venue','venueid','journal','doi'
                        ]

    # test = [key for key in new_list if key in initial_list_] 
    print("keys_initial_list :\n",keys_initial_list)
    valid_keys = list(filter(lambda x: x in keys_initial_list,content_dict.keys()))
    print("\nvalid_keys :\n",valid_keys)
    
    key_for_all_RS = key_for_all_rs
    title_rs = content_dict['title'] if 'title' in valid_keys else ""
    authors_rs = content_dict['authors'] if 'authors' in valid_keys else ""
    article_type = "Replication"

    keywords = content_dict['keywords'] if 'keywords' in valid_keys else ""
    abstract = content_dict['abstract'] if 'abstract' in valid_keys else ""
    pdf_url_rs = content_dict['pdf'] if 'pdf' in valid_keys else ""
    code_url_rs = ""
    org_paper_url = content_dict['paper_url'] if 'paper_url' in valid_keys else ""
    
    doi_rs = content_dict['doi'] if 'doi' in valid_keys else ""
    code_doi_rs = ''
    doi_org = ''
    
    venue_name_rs = 'MLRC'
    challenge_year_rs = '2022'
    review_url_rs = row["re_details"]

    journal_rs = ''
    volume_journal_rs = ''
    issue_journal_rs = ''
    published_year_journal_rs = ''

    domain = 'ML Reproducibility Challenge 2021'
    entry_type = 'article'  
            
    dict_ = dict()

    for key in all_meta_data_keys:
        dict_.update({key:eval(key)})

    row_items.append(dict_)          
    
df = pd.DataFrame(row_items)
df.to_csv("RS_ALL_IN_ONE_2022.csv", index = False)

1
keys_initial_list :
 ['title', 'authors', 'TL;DR', 'keywords', 'abstract', 'pdf', 'paper_url', 'paper_venue', 'venue_name', 'paperhash', 'venue', 'venueid', 'journal', 'doi']

valid_keys :
 ['title', 'authors', 'keywords', 'TL;DR', 'abstract', 'pdf', 'paper_url', 'paper_venue', 'paperhash', 'venue', 'venueid', 'journal', 'doi']
2
keys_initial_list :
 ['title', 'authors', 'TL;DR', 'keywords', 'abstract', 'pdf', 'paper_url', 'paper_venue', 'venue_name', 'paperhash', 'venue', 'venueid', 'journal', 'doi']

valid_keys :
 ['title', 'authors', 'keywords', 'TL;DR', 'abstract', 'pdf', 'paper_url', 'paper_venue', 'paperhash', 'venue', 'venueid', 'journal', 'doi']
3
keys_initial_list :
 ['title', 'authors', 'TL;DR', 'keywords', 'abstract', 'pdf', 'paper_url', 'paper_venue', 'venue_name', 'paperhash', 'venue', 'venueid', 'journal', 'doi']

valid_keys :
 ['title', 'authors', 'keywords', 'TL;DR', 'abstract', 'pdf', 'paper_url', 'paper_venue', 'paperhash', 'venue', 'venueid', 'journal', 'doi']
4
ke

26
keys_initial_list :
 ['title', 'authors', 'TL;DR', 'keywords', 'abstract', 'pdf', 'paper_url', 'paper_venue', 'venue_name', 'paperhash', 'venue', 'venueid', 'journal', 'doi']

valid_keys :
 ['title', 'authors', 'keywords', 'abstract', 'pdf', 'paper_url', 'paper_venue', 'paperhash', 'venue', 'venueid', 'journal', 'doi']
27
keys_initial_list :
 ['title', 'authors', 'TL;DR', 'keywords', 'abstract', 'pdf', 'paper_url', 'paper_venue', 'venue_name', 'paperhash', 'venue', 'venueid', 'journal', 'doi']

valid_keys :
 ['title', 'authors', 'keywords', 'TL;DR', 'abstract', 'pdf', 'paper_url', 'paper_venue', 'paperhash', 'venue', 'venueid', 'journal', 'doi']
28
keys_initial_list :
 ['title', 'authors', 'TL;DR', 'keywords', 'abstract', 'pdf', 'paper_url', 'paper_venue', 'venue_name', 'paperhash', 'venue', 'venueid', 'journal', 'doi']

valid_keys :
 ['title', 'authors', 'keywords', 'abstract', 'pdf', 'paper_url', 'paper_venue', 'paperhash', 'TL;DR', 'venue', 'venueid', 'journal', 'doi']
29
keys_in

## ``Automated original paper url extraction``

In [9]:
df_xlsx = pd.read_excel('RS_ALL_IN_ONE_metadata.xlsx')

df_xlsx.head()

,key_for_all_RS,title_rs,authors_rs,article_type,keywords,abstract,pdf_url_rs,code_url_rs,org_paper_url,doi_rs,...,challenge_year_rs,review_url_rs,journal_rs,volume_journal_rs,issue_journal_rs,published_year_journal_rs,domain,entry_type,rs_score,rs_comment
0,RS_001_MLRC_2022_01,[Re] $\mathcal{G}$-Mixup: Graph Data Augmentat...,"['Ermin Omeragic', 'Vuk ÄuranoviÄ‡']",Replication,"['rescience c', 'machine learning', 'data augm...",Scope of Reproducibility\nThis paper presents ...,/pdf/8c87cb1e84e1482826c40a3b0c43928eaef747f3.pdf,NaN,https://proceedings.mlr.press/v162/han22c.html,https://www.doi.org/10.5281/zenodo.8173650,...,2022,https://openreview.net/forum?id=XxUIomN-ndH,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN
1,RS_002_MLRC_2022_02,[Re] Exact Feature Distribution Matching for A...,"['Mert Erkol', 'Furkan KÄ±nlÄ±', 'BarÄ±ÅŸ Ã–zc...",Replication,"['feature distribution matching', 'style trans...",Reproducibility Summary:\n\nIn this reproducib...,/pdf/7bcb577c2a46db29c48234a5b72368053c7ebed3.pdf,NaN,https://openaccess.thecvf.com/content/CVPR2022...,https://www.doi.org/10.5281/zenodo.8173652,...,2022,https://openreview.net/forum?id=a5_hbZf0NB&not...,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN
2,RS_003_MLRC_2022_03,[Re] End-to-end Algorithm Synthesis with Recur...,"['Sean Michael McLeish', 'Long Tran-Thanh']",Replication,"['rescience c', 'Algorithmic Reasoning', 'Deep...","Scope of Reproducibility:\nIn this report, we ...",/pdf/07d5d68b5873d779bd1fd8c95b9767cb57fe0bc4.pdf,NaN,https://arxiv.org/pdf/2202.05826.pdf,https://www.doi.org/10.5281/zenodo.8173654,...,2022,https://openreview.net/forum?id=WaZB4pUVTi,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN
3,RS_004_MLRC_2022_04,[Re] Label-Free Explainability for Unsupervise...,"['Eric Langezaal', 'Jesse Belleman', 'Joeri No...",Replication,"['Reproducibility', 'label-free', 'unsupervise...",Scope of Reproducibility â€” This study is an ...,/pdf/7fde4f12c675150699fc376cd097d5b9dad0b4d4.pdf,NaN,https://proceedings.mlr.press/v162/crabbe22a.html,https://www.doi.org/10.5281/zenodo.8173656,...,2022,https://openreview.net/forum?id=bBVZ3pY4z8p,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN
4,RS_005_MLRC_2022_05,[Re] Exploring the Representation of Word Mean...,"['Matteo Brivio', 'Cagri Coltekin']",Replication,"['BERT', 'fastText', 'embeddings', 'language m...",This report summarizes our efforts to reproduc...,/pdf/78430c5af33bc892d852b49a5a6e93abeb314c6f.pdf,NaN,https://aclanthology.org/2021.acl-long.281/,https://www.doi.org/10.5281/zenodo.8173658,...,2022,https://openreview.net/forum?id=Od5dD58libt,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN


In [11]:
new_df_with_doi = df_xlsx.copy()

for index, row in df_xlsx.iterrows():
    print(row['key_for_all_RS'])
    if str(row['review_url_rs']) != 'nan':
        if index>116:
            print(row['review_url_rs'])
            pass
        else:
            page = requests.get(row['review_url_rs'].strip())
            soup = BeautifulSoup(page.content, "html.parser")

            # Meta-Data extraction
            details = soup.find(id="__NEXT_DATA__")
            full_dict = json.loads(details.text)
            content_dict = full_dict['props']['pageProps']['forumNote']['content']

            if "DO EXTRACT FROM" not in str(row['org_paper_url']):
                 print("A value is already there")
            else:
                new_df_with_doi.at[index, 'org_paper_url'] = content_dict.get('paper_url')   
    else:
        print(row['key_for_all_RS'], '----- nan ISSUE')
        
new_df_with_doi.reset_index(drop=True)    

new_df_with_doi.to_excel("RS_ALL_IN_ONE_metadata_with_org_paper_URL.xlsx",index=False)  
new_df_with_doi.head(50)

RS_001_MLRC_2022_01
A value is already there
RS_002_MLRC_2022_02
A value is already there
RS_003_MLRC_2022_03
A value is already there
RS_004_MLRC_2022_04
A value is already there
RS_005_MLRC_2022_05
A value is already there
RS_006_MLRC_2022_06
A value is already there
RS_007_MLRC_2022_07
A value is already there
RS_008_MLRC_2022_08
A value is already there
RS_009_MLRC_2022_09
A value is already there
RS_010_MLRC_2022_10
A value is already there
RS_011_MLRC_2022_11
A value is already there
RS_012_MLRC_2022_12
A value is already there
RS_013_MLRC_2022_13
A value is already there
RS_014_MLRC_2022_14
A value is already there
RS_015_MLRC_2022_15
A value is already there
RS_016_MLRC_2022_16
A value is already there
RS_017_MLRC_2022_17
A value is already there
RS_018_MLRC_2022_18
A value is already there
RS_019_MLRC_2022_19
A value is already there
RS_020_MLRC_2022_20
A value is already there
RS_021_MLRC_2022_21
A value is already there
RS_022_MLRC_2022_22
A value is already there
RS_023_MLR

,key_for_all_RS,title_rs,authors_rs,article_type,keywords,abstract,pdf_url_rs,code_url_rs,org_paper_url,doi_rs,...,challenge_year_rs,review_url_rs,journal_rs,volume_journal_rs,issue_journal_rs,published_year_journal_rs,domain,entry_type,rs_score,rs_comment
0,RS_001_MLRC_2022_01,[Re] $\mathcal{G}$-Mixup: Graph Data Augmentat...,"['Ermin Omeragic', 'Vuk ÄuranoviÄ‡']",Replication,"['rescience c', 'machine learning', 'data augm...",Scope of Reproducibility\nThis paper presents ...,/pdf/8c87cb1e84e1482826c40a3b0c43928eaef747f3.pdf,NaN,https://proceedings.mlr.press/v162/han22c.html,https://www.doi.org/10.5281/zenodo.8173650,...,2022,https://openreview.net/forum?id=XxUIomN-ndH,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN
1,RS_002_MLRC_2022_02,[Re] Exact Feature Distribution Matching for A...,"['Mert Erkol', 'Furkan KÄ±nlÄ±', 'BarÄ±ÅŸ Ã–zc...",Replication,"['feature distribution matching', 'style trans...",Reproducibility Summary:\n\nIn this reproducib...,/pdf/7bcb577c2a46db29c48234a5b72368053c7ebed3.pdf,NaN,https://openaccess.thecvf.com/content/CVPR2022...,https://www.doi.org/10.5281/zenodo.8173652,...,2022,https://openreview.net/forum?id=a5_hbZf0NB&not...,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN
2,RS_003_MLRC_2022_03,[Re] End-to-end Algorithm Synthesis with Recur...,"['Sean Michael McLeish', 'Long Tran-Thanh']",Replication,"['rescience c', 'Algorithmic Reasoning', 'Deep...","Scope of Reproducibility:\nIn this report, we ...",/pdf/07d5d68b5873d779bd1fd8c95b9767cb57fe0bc4.pdf,NaN,https://arxiv.org/pdf/2202.05826.pdf,https://www.doi.org/10.5281/zenodo.8173654,...,2022,https://openreview.net/forum?id=WaZB4pUVTi,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN
3,RS_004_MLRC_2022_04,[Re] Label-Free Explainability for Unsupervise...,"['Eric Langezaal', 'Jesse Belleman', 'Joeri No...",Replication,"['Reproducibility', 'label-free', 'unsupervise...",Scope of Reproducibility â€” This study is an ...,/pdf/7fde4f12c675150699fc376cd097d5b9dad0b4d4.pdf,NaN,https://proceedings.mlr.press/v162/crabbe22a.html,https://www.doi.org/10.5281/zenodo.8173656,...,2022,https://openreview.net/forum?id=bBVZ3pY4z8p,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN
4,RS_005_MLRC_2022_05,[Re] Exploring the Representation of Word Mean...,"['Matteo Brivio', 'Cagri Coltekin']",Replication,"['BERT', 'fastText', 'embeddings', 'language m...",This report summarizes our efforts to reproduc...,/pdf/78430c5af33bc892d852b49a5a6e93abeb314c6f.pdf,NaN,https://aclanthology.org/2021.acl-long.281/,https://www.doi.org/10.5281/zenodo.8173658,...,2022,https://openreview.net/forum?id=Od5dD58libt,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN
5,RS_006_MLRC_2022_06,[Re] Intriguing Properties of Contrastive Losses,"['Luca Marini', 'Mohamad Nabeel', 'Alexandre L...",Replication,"['self-supervised learning', 'contrastive lear...",Reproducibility Summary\n\nScope of Reproducib...,/pdf/fc591d20ed7c340bbe470789beb95d4cde81e7d5.pdf,NaN,https://proceedings.neurips.cc/paper/2021/hash...,https://www.doi.org/10.5281/zenodo.8173662,...,2022,https://openreview.net/forum?id=gb71irTNN7,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN
6,RS_007_MLRC_2022_07,[Re] Bandit Theory and Thompson Sampling-guide...,['Luka Å½ontar'],Replication,"['Optimization', 'bandit learning', 'Thompson ...",The paper presents a novel DE approach using T...,/pdf/cf9062c0ba6651cb416547df2c1e7e7379d572a9.pdf,NaN,https://neurips.cc/Conferences/2022/ScheduleMu...,https://www.doi.org/10.5281/zenodo.8173664,...,2022,https://openreview.net/forum?id=NE_x1dpz-Q,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN
7,RS_008_MLRC_2022_08,[Re] Hypergraph-Induced Semantic Tuplet Loss f...,"['Jicheng Yuan', 'Danh Le Phuoc']",Replication,"['Hypergraph', 'Deep Metric Learning', 'Semant...",Reproducibility Summary\n\nScope of Reproducib...,/pdf/9a228c78b9d481df388578a8e5d78a90412b9891.pdf,NaN,https://openaccess.thecv

## ``original paper doi search (manual process )``

In [16]:
df_url = pd.read_excel('RS_ALL_IN_ONE_metadata.xlsx',sheet_name="Sheet1")

df_url.head(50)

,key_for_all_RS,title_rs,authors_rs,article_type,keywords,abstract,pdf_url_rs,code_url_rs,org_paper_url,doi_rs,...,challenge_year_rs,review_url_rs,journal_rs,volume_journal_rs,issue_journal_rs,published_year_journal_rs,domain,entry_type,rs_score,rs_comment
0,RS_001_MLRC_2022_01,[Re] $\mathcal{G}$-Mixup: Graph Data Augmentat...,"['Ermin Omeragic', 'Vuk ÄuranoviÄ‡']",Replication,"['rescience c', 'machine learning', 'data augm...",Scope of Reproducibility\nThis paper presents ...,/pdf/8c87cb1e84e1482826c40a3b0c43928eaef747f3.pdf,NaN,https://proceedings.mlr.press/v162/han22c.html,https://www.doi.org/10.5281/zenodo.8173650,...,2022,https://openreview.net/forum?id=XxUIomN-ndH,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN
1,RS_002_MLRC_2022_02,[Re] Exact Feature Distribution Matching for A...,"['Mert Erkol', 'Furkan KÄ±nlÄ±', 'BarÄ±ÅŸ Ã–zc...",Replication,"['feature distribution matching', 'style trans...",Reproducibility Summary:\n\nIn this reproducib...,/pdf/7bcb577c2a46db29c48234a5b72368053c7ebed3.pdf,NaN,https://openaccess.thecvf.com/content/CVPR2022...,https://www.doi.org/10.5281/zenodo.8173652,...,2022,https://openreview.net/forum?id=a5_hbZf0NB&not...,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN
2,RS_003_MLRC_2022_03,[Re] End-to-end Algorithm Synthesis with Recur...,"['Sean Michael McLeish', 'Long Tran-Thanh']",Replication,"['rescience c', 'Algorithmic Reasoning', 'Deep...","Scope of Reproducibility:\nIn this report, we ...",/pdf/07d5d68b5873d779bd1fd8c95b9767cb57fe0bc4.pdf,NaN,https://arxiv.org/pdf/2202.05826.pdf,https://www.doi.org/10.5281/zenodo.8173654,...,2022,https://openreview.net/forum?id=WaZB4pUVTi,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN
3,RS_004_MLRC_2022_04,[Re] Label-Free Explainability for Unsupervise...,"['Eric Langezaal', 'Jesse Belleman', 'Joeri No...",Replication,"['Reproducibility', 'label-free', 'unsupervise...",Scope of Reproducibility â€” This study is an ...,/pdf/7fde4f12c675150699fc376cd097d5b9dad0b4d4.pdf,NaN,https://proceedings.mlr.press/v162/crabbe22a.html,https://www.doi.org/10.5281/zenodo.8173656,...,2022,https://openreview.net/forum?id=bBVZ3pY4z8p,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN
4,RS_005_MLRC_2022_05,[Re] Exploring the Representation of Word Mean...,"['Matteo Brivio', 'Cagri Coltekin']",Replication,"['BERT', 'fastText', 'embeddings', 'language m...",This report summarizes our efforts to reproduc...,/pdf/78430c5af33bc892d852b49a5a6e93abeb314c6f.pdf,NaN,https://aclanthology.org/2021.acl-long.281/,https://www.doi.org/10.5281/zenodo.8173658,...,2022,https://openreview.net/forum?id=Od5dD58libt,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN
5,RS_006_MLRC_2022_06,[Re] Intriguing Properties of Contrastive Losses,"['Luca Marini', 'Mohamad Nabeel', 'Alexandre L...",Replication,"['self-supervised learning', 'contrastive lear...",Reproducibility Summary\n\nScope of Reproducib...,/pdf/fc591d20ed7c340bbe470789beb95d4cde81e7d5.pdf,NaN,https://proceedings.neurips.cc/paper/2021/hash...,https://www.doi.org/10.5281/zenodo.8173662,...,2022,https://openreview.net/forum?id=gb71irTNN7,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN
6,RS_007_MLRC_2022_07,[Re] Bandit Theory and Thompson Sampling-guide...,['Luka Å½ontar'],Replication,"['Optimization', 'bandit learning', 'Thompson ...",The paper presents a novel DE approach using T...,/pdf/cf9062c0ba6651cb416547df2c1e7e7379d572a9.pdf,NaN,https://neurips.cc/Conferences/2022/ScheduleMu...,https://www.doi.org/10.5281/zenodo.8173664,...,2022,https://openreview.net/forum?id=NE_x1dpz-Q,NaN,NaN,NaN,NaN,ML Reproducibility Challenge 2021,article,NaN,NaN
7,RS_008_MLRC_2022_08,[Re] Hypergraph-Induced Semantic Tuplet Loss f...,"['Jicheng Yuan', 'Danh Le Phuoc']",Replication,"['Hypergraph', 'Deep Metric Learning', 'Semant...",Reproducibility Summary\n\nScope of Reproducib...,/pdf/9a228c78b9d481df388578a8e5d78a90412b9891.pdf,NaN,https://openaccess.thecv

In [17]:
for index, row in df_url.iterrows():
    
    print(row['key_for_all_RS'])
    print(row['org_paper_url'])
    print("\n\n\n")

RS_001_MLRC_2022_01
https://proceedings.mlr.press/v162/han22c.html




RS_002_MLRC_2022_02
https://openaccess.thecvf.com/content/CVPR2022/papers/Zhang_Exact_Feature_Distribution_Matching_for_Arbitrary_Style_Transfer_and_Domain_CVPR_2022_paper.pdf




RS_003_MLRC_2022_03
https://arxiv.org/pdf/2202.05826.pdf




RS_004_MLRC_2022_04
https://proceedings.mlr.press/v162/crabbe22a.html




RS_005_MLRC_2022_05
https://aclanthology.org/2021.acl-long.281/




RS_006_MLRC_2022_06
https://proceedings.neurips.cc/paper/2021/hash/628f16b29939d1b060af49f66ae0f7f8-Abstract.html




RS_007_MLRC_2022_07
https://neurips.cc/Conferences/2022/ScheduleMultitrack?event=53784




RS_008_MLRC_2022_08
https://openaccess.thecvf.com/content/CVPR2022/papers/Lim_Hypergraph-Induced_Semantic_Tuplet_Loss_for_Deep_Metric_Learning_CVPR_2022_paper.pdf




RS_009_MLRC_2022_09
https://openreview.net/forum?id=ao30zaT3YL




RS_010_MLRC_2022_10
https://doi.org/10.1007/978-3-031-19775-8_26




RS_011_MLRC_2022_11
https://procee

## ``parse pdfs for rs_comment``

In [3]:
file_names = glob.glob("RS_ALL_148/*.pdf")
file_names

['RS_ALL_148\\RS_001_MLRC_2022_01.pdf',
 'RS_ALL_148\\RS_002_MLRC_2022_02.pdf',
 'RS_ALL_148\\RS_003_MLRC_2022_03.pdf',
 'RS_ALL_148\\RS_004_MLRC_2022_04.pdf',
 'RS_ALL_148\\RS_005_MLRC_2022_05.pdf',
 'RS_ALL_148\\RS_006_MLRC_2022_06.pdf',
 'RS_ALL_148\\RS_007_MLRC_2022_07.pdf',
 'RS_ALL_148\\RS_008_MLRC_2022_08.pdf',
 'RS_ALL_148\\RS_009_MLRC_2022_09.pdf',
 'RS_ALL_148\\RS_010_MLRC_2022_10.pdf',
 'RS_ALL_148\\RS_011_MLRC_2022_11.pdf',
 'RS_ALL_148\\RS_012_MLRC_2022_12.pdf',
 'RS_ALL_148\\RS_013_MLRC_2022_13.pdf',
 'RS_ALL_148\\RS_014_MLRC_2022_14.pdf',
 'RS_ALL_148\\RS_015_MLRC_2022_15.pdf',
 'RS_ALL_148\\RS_016_MLRC_2022_16.pdf',
 'RS_ALL_148\\RS_017_MLRC_2022_17.pdf',
 'RS_ALL_148\\RS_018_MLRC_2022_18.pdf',
 'RS_ALL_148\\RS_019_MLRC_2022_19.pdf',
 'RS_ALL_148\\RS_020_MLRC_2022_20.pdf',
 'RS_ALL_148\\RS_021_MLRC_2022_21.pdf',
 'RS_ALL_148\\RS_022_MLRC_2022_22.pdf',
 'RS_ALL_148\\RS_023_MLRC_2022_23.pdf',
 'RS_ALL_148\\RS_024_MLRC_2022_24.pdf',
 'RS_ALL_148\\RS_025_MLRC_2022_25.pdf',


In [1]:
from pdfminer.high_level import extract_pages

for page_layout in extract_pages("RS_ALL_148\\RS_002_MLRC_2022_02.pdf"):
    for element in page_layout:
        print(element)

<LTTextBoxHorizontal(0) 11.683,725.909,94.604,734.876 'R E S C I E N C E C\n'>
<LTTextBoxHorizontal(1) 127.559,693.286,486.414,744.746 'Replication / ML Reproducibility Challenge 2022\n[Re] Exact Feature Distribution Matching for Arbitrary\nStyle Transfer and Domain Generalization\n'>
<LTTextBoxHorizontal(2) 127.410,650.243,362.549,672.531 'Mert Erkol1, ID , Furkan Kınlı1, ID , Barış Özcan1, ID , and Furkan Kıraç1, ID\n1Ozyegin University, Vision and Graphics Laboratory, Istanbul, Turkey\n'>
<LTTextBoxHorizontal(3) 54.396,577.323,106.961,613.686 'Edited by\nKoustuv Sinha,\nMaurits Bleeker,\nSamarth Bhargav\n'>
<LTTextBoxHorizontal(4) 53.719,548.929,106.307,566.364 'Received\n04 February 2023\n'>
<LTTextBoxHorizontal(5) 67.869,520.536,106.312,537.970 'Published\n20 July 2023\n'>
<LTTextBoxHorizontal(6) 29.583,492.142,106.327,509.577 'DOI\n10.5281/zenodo.8173652\n'>
<LTTextBoxHorizontal(7) 127.559,579.150,268.023,593.496 'Reproducibility Summary\n'>
<LTTextBoxHorizontal(8) 127.559,471.98

<LTTextBoxHorizontal(0) 127.559,782.258,375.715,789.232 '[Re] Exact Feature Distribution Matching for Arbitrary Style Transfer and Domain Generalization\n'>
<LTTextBoxHorizontal(1) 129.161,556.793,508.644,565.768 'Figure 3. Illustration of style interpolation between a single content image and four style images.\n'>
<LTTextBoxHorizontal(2) 127.335,511.971,510.471,542.854 'Table 1. Average running time of prior methods and proposed strategy used in AST on a 512px\nimage. Note that the compared methods run on a single Tesla V100, while our measurement has\nbeen done on a single RTX 2080Ti.\n'>
<LTTextBoxHorizontal(3) 133.594,477.330,504.210,499.647 'Method Gatys et al. [24] CMD [10] HM AdaIN [4] EFDM [1] EFDM (ours)\n0.33\nTime (s)\n'>
<LTTextBoxHorizontal(4) 462.517,477.330,490.064,487.293 '0.0043\n'>
<LTTextBoxHorizontal(5) 348.968,477.330,376.514,487.293 '0.0038\n'>
<LTTextBoxHorizontal(6) 402.158,477.330,429.705,487.293 '0.0039\n'>
<LTTextBoxHorizontal(7) 268.699,477.330,291.334,487.

<LTTextBoxHorizontal(0) 127.559,782.258,375.715,789.232 '[Re] Exact Feature Distribution Matching for Arbitrary Style Transfer and Domain Generalization\n'>
<LTTextBoxHorizontal(1) 127.559,732.243,510.232,754.161 'to add t‐SNE visualizations to this report, as in the original paper, due to the lack of\nclarity in the documentation of its script.\n'>
<LTTextBoxHorizontal(2) 107.758,702.365,298.757,714.320 '5.3 Communication with original authors\n'>
<LTTextBoxHorizontal(3) 127.081,671.907,510.233,693.825 'We were in contact with the authors, and asked for the original results as JPEG files to\nprepare the figures in this report.\n'>
<LTTextBoxHorizontal(4) 127.559,636.213,189.922,650.559 'References\n'>
<LTTextBoxHorizontal(5) 131.841,591.571,137.992,600.538 '2.\n'>
<LTTextBoxHorizontal(6) 131.841,558.694,137.992,567.661 '3.\n'>
<LTTextBoxHorizontal(7) 131.841,613.489,137.992,622.456 '1.\n'>
<LTTextBoxHorizontal(8) 131.841,471.023,511.564,622.456 'Y. Zhang, M. Li, R. Li, K. Jia, and L. 

In [19]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer

import glob

# https://github.com/pdfminer/pdfminer.six/blob/22f90521b823ac5a22785d1439a64c7bdf2c2c6d/pdfminer/high_level.py#L126
for page_layout in extract_pages("RS_ALL_148\\RS_003_MLRC_2022_03.pdf",maxpages=1):
    for element in page_layout:
        if isinstance(element, LTTextContainer):
            el_text = element.get_text()
            
            if el_text.split(' ')[0].lower() == 'results':
                print(el_text)
                break
            else:
                print("NOT FOUND")
    break

NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
Results — We verify the authors’ claims by replicating the experiments presented in [1].
All of our experiments show identical results to the ones presented in [1], apart from per‐
turbation testing for which we provide an additional in depth analysis. We also provide
an analysis of the new alpha hyperparameter and a hyperparameter search.



In [38]:
#for 2021 and 2020

flag = 0
for page_layout in extract_pages("RS_ALL_148\\RS_056_MLRC_2021_11.pdf",maxpages=1):
    
    for element in page_layout:
        if isinstance(element, LTTextContainer):
            if flag == 1 : 
                print(element.get_text())
                break

            el_text = element.get_text()
            
            if 'results' in el_text.lower():
                print(element)
                flag = 1
    break

<LTTextBoxHorizontal(13) 107.758,346.972,162.349,358.927 '1.3 Results\n'>
Overall we find the original results to be reproducible; transformation policies found
using Gao et al.1’s method can defend against gradient reconstruction attacks, and these
transformations have negligible impact on training efficiency and model accuracy. How‐
ever we do not observe the reported correlation between the proposed privacy‐score Spri
and reconstruction PSNR. We also find that the degree of protection differs greatly from
image to image, with poor protection in the worst case.



In [42]:
flag = 0
for page_layout in extract_pages("RS_ALL_148\\RS_124_NeurIPS_2019_07.pdf",maxpages=1):
    
    for element in page_layout:
        if isinstance(element, LTTextContainer):
            if flag == 1 : 
                print(element.get_text())
                break

            el_text = element.get_text()
            
            if 'abstract' in el_text.lower():
                print(element)
                flag = 1
    break

<LTTextBoxHorizontal(8) 301.332,608.970,336.464,617.937 'Abstract\n'>
We reproduce the work in Zero-shot Knowledge Transfer via Adversarial Belief
Matching, which describes a novel approach for knowledge transfer. A teacher net-
work trained on real samples distills knowledge to a student network that is trained
solely on pseudo data extracted from a generator network, with the student trying
to mimic the teacher’s outputs on these datapoints. To this end, we additionally
re-implement Wide Residual Networks which are used as the main framework for
both teacher and student networks and train them from scratch on CIFAR10 and
SVHN. We compare the results of the proposed method with a few-shot knowledge
distillation attention transfer setting implemented and trained from scratch. We
suggest an approach for further exploitation of the learnt mechanics of the gener-
ator network in the zero-shot setting, which operates on top of the main method,
and brieﬂy discuss the beneﬁts and drawbacks o

In [65]:
file_names = glob.glob("RS_ALL_148/*.pdf")

df_excel = pd.read_excel('RS_ALL_IN_ONE_metadata_TEST.xlsx',sheet_name="Sheet1")


def extract_1st_page_results_2022(filename,max_pages = 1):
    for page_layout in extract_pages(file,maxpages=max_pages):
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                el_text = element.get_text()

                if el_text.split(' ')[0].lower() == 'results':
                    print(el_text)
                    return el_text
                else:
                    print("NOT FOUND")

                    
def extract_1st_page_results_2020_2021(filename,max_pages = 1):
    flag = 0
    for page_layout in extract_pages(filename,maxpages=1):

        for element in page_layout:
            if isinstance(element, LTTextContainer):
                if flag == 1 : 
                    print(element.get_text())
                    return element.get_text()

                el_text = element.get_text()

                if 'results' in el_text.lower() and len(el_text.split())<6:
                    print(element)
                    flag = 1 # returning the next element values
                elif el_text.split()[0].lower() == 'results':
                    return element.get_text()
                

def extract_1st_page_results_NeurIPS_2019(filename,max_pages = 1):
    flag = 0
    for page_layout in extract_pages(filename,maxpages=1):

        for element in page_layout:
            if isinstance(element, LTTextContainer):
                if flag == 1 : 
                    print(element.get_text())
                    return element.get_text()

                el_text = element.get_text()

                if 'abstract' in el_text.lower():
                    print(element)
                    flag = 1 # returning the next element values


for index, file in enumerate(file_names):

    reults_text = "NOT EXTRACTED"
    
    rs_key = file.split('\\')[1].replace('.pdf','')
    
    required_index = df_excel.loc[df_excel["key_for_all_RS"] == rs_key].index[0]
    print(required_index)
    print(index, " --- ",  rs_key)
    
    if "MLRC_2022" in file:
        reults_text =  extract_1st_page_results_2022(file)
        
        if reults_text == '' or reults_text is None:
            df_excel.at[required_index, 'rs_comment'] = "NOT EXTRACTED"
        else:
            df_excel.at[required_index, 'rs_comment'] = reults_text
        
        continue
    
    elif "MLRC_2021" in file or "MLRC_2020" in file:
        reults_text =  extract_1st_page_results_2020_2021(file)   

        if reults_text == '' or reults_text is None:
            df_excel.at[required_index, 'rs_comment'] = "NOT EXTRACTED"
        else:
            df_excel.at[required_index, 'rs_comment'] = reults_text        
        
        continue
        
    elif "NeurIPS_2019" in file:
        reults_text =  extract_1st_page_results_NeurIPS_2019(file) 

        if reults_text == '' or reults_text is None:
            df_excel.at[required_index, 'rs_comment'] = "NOT EXTRACTED"
        else:
            df_excel.at[required_index, 'rs_comment'] = reults_text        
            
        continue

    else:
        pass
        
    print("\n\n\n")
    
df_excel.to_excel("RS_ALL_IN_ONE_metadata_with_rs_comment.xlsx",index=False)  


0
0  ---  RS_001_MLRC_2022_01
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
Results — Due to many missing implementation details, we were not able to reproduce all
of the original results. Some claims can be supported by our results, but most results are
very vague. Even though the new method outperforms the baselines in certain scenarios,
we find that the superiority of the method is not as strong as presented in the original
paper.

1
1  ---  RS_002_MLRC_2022_02
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
Results — We have reproduced the experiments done on two selected tasks, and com‐
pared their results with the reported results. Although our experimental results are not
identical to the reported ones, we can validate the claims made by the original study
according to these results.

2
2  ---  RS_003_MLRC_2022_03
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
N

NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
Results — The results that we got were very coherent with the ones exposed in the pa‐
per. Although we were not able to run many experiments with large models, we got a
big insight into the topic. Something that is somehow negligible in theory, combined
with finite‐precision arithmetics and the bad election of the subgradient, might lead to
chaotic behaviour. This gives big support to the default values and gives a solid answer
to the question: What would it be the best subgradient election? The theory says: it
doesn’t matter, yet it does.

17
17  ---  RS_018_MLRC_2022_18
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
Results — The claim that Shifty guarantees fairness with high confidence is strongly
confirmed by the reproduction results of this study. It was also found in this repro‐
ducibility study that Shifty achieves accuracy sco

33
33  ---  RS_034_MLRC_2022_34
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
Results — We verify their theoretical claims that G‐Mixup indeed leads to a mixture of
“key graph topologies.” We reproduce their experimental results on classification accu‐
racy for REDDIT‐B to within about 6% of the reported value, and for IMDB‐B to within
about 2% of the reported value. However, our experimental results do not provide sta‐
tistically significant evidence to support the paper’s experimental claims that G‐Mixup
improves the performance of GNNs or performs better than other graph data augmen‐
tation methods.

34
34  ---  RS_035_MLRC_2022_35
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
Results — Our results are consistent with the claim that the censored text (without swear
words) is less often classified as hate speech, offensive, or abusive than the same text
with swear words. Howeve

<LTTextBoxHorizontal(13) 127.559,301.592,162.349,313.547 'Results\n'>
The results in the original paper regarding different objective functions were repro‐
duced within a margin of error. Also, income redistribution is able to reduce wage
inequality, albeit to a lesser degree. The objective functions appear to be sensitive to
the neighbourhood selection mechanism. While the results of the rider‐fairness objec‐
tive functions are maintained, performance of the driver‐fairness objective functions
declines. There appear to be only small differences in service rates between ethnicities,
while rider‐side fairness seems to mitigate inequalities the most. However, this is only
achieved by worsening the service for well‐served neighbourhoods instead of improving
it for underserved ones.

51
51  ---  RS_052_MLRC_2021_07
<LTTextBoxHorizontal(14) 107.758,237.720,162.349,249.675 '1.3 Results\n'>
Our findings diverge substantially from the results reported in the original paper. In
particular, in o

<LTTextBoxHorizontal(12) 127.559,272.077,162.349,284.032 'Results\n'>
Most of our results closely match the reported results in the original paper for the Rotated‐
MNIST [1], Fashion‐MNIST [2], PACS [3, 4], and Chest‐Xray [5] datasets. However, in
some cases, as described later, we obtained better results quantitatively than the ones
reported in the paper. By investigating the root cause of such mismatches, we provide
a possible reason to avoid such a gap. We performed additional experiments by making
necessary modifications for the Rotated‐MNIST and Rotated Fashion‐MNIST dataset. In

63
63  ---  RS_064_MLRC_2021_19
<LTTextBoxHorizontal(17) 127.559,320.521,162.349,332.476 'Results\n'>
Our results followed similar patterns as those of the authors, which backs up their
claims regarding the attacks. However, our results did slightly deviate from their re‐
sults, meaning the original paper has some reproducibility issues in the context of our
experimental setup.

64
64  ---  RS_065_MLRC_20

<LTTextBoxHorizontal(13) 127.559,322.513,162.349,334.468 'Results\n'>
We reproduced the accuracy for the SPT planner module to within 14.7% of reported
value, which, while outperforming the baselines [2] [3] in select cases, fails to support
the paper’s conclusion that it outperforms the baselines. However, we achieve a similar
drop‐off in accuracy in percentage points over different model settings. We suspect that
the vagueness in the accuracy metric leads to the absolute difference of 14.7% despite
the paper being reproducible. We further improve the reproduced figures by increasing
model complexity. The Mapper module’s accuracy could not be tested.

79
79  ---  RS_080_MLRC_2021_35
<LTTextBoxHorizontal(13) 127.559,310.558,162.349,322.513 'Results\n'>
In general, we are able to reproduce the results of Hyder et al. [1]. Because of the hy‐
perparameter search, we are certain that the results are not cherry‐picked and mostly
reproducible using the authors’ implementation of the algorith

<LTTextBoxHorizontal(13) 127.559,336.637,162.349,348.592 'Results\n'>
We were able to reproduce their results using their code, yielding mostly similar results.
TGT generally outperforms DBM, especially when explanations use few features. TGT
is consistent in terms of the features to which it attributes cluster differences, across
different sparsity levels. TGT matches real patterns in data. When extending the types of
functions used for explanations, performance did not improve significantly, suggesting
translations make for adequate explanations. However, the scaling extension shows
promising performance on the modified synthetic data to recover the original signal.

94
94  ---  RS_095_MLRC_2020_02
<LTTextBoxHorizontal(13) 127.559,289.636,162.349,301.591 'Results\n'>
We reproduce the claims of both papers by conducting several experiments in the UAVA
dataset [3]. The integration of a differentiable geometric module within an keypoint-
based object pose estimation model improved its p

111
111  ---  RS_112_MLRC_2020_19
<LTTextBoxHorizontal(14) 127.559,313.315,162.349,325.270 'Results\n'>
During the study, we were not able to reproduce the method due to a conceptual mis-
interpretation of ours regarding the authorsʼ adaption of the Transformer [2]. However,
the publicly available implementation helped us answering our questions and proved
its validity during our experiments on different datasets. Additionally, we compared the
papersʼ temporal attention encoder to our adaption of it, which we came across while
we were trying to reimplement and grasp the authorsʼ ideas.

112
112  ---  RS_113_MLRC_2020_20
<LTTextBoxHorizontal(13) 127.559,370.883,162.349,382.838 'Results\n'>
In contrast to the first claim, we have discovered that for most of the architectures, re-
construction errors for the attacks are quite low, which means that in our models the
first claim is not supported. We also found that for most of the models, the classifica-
tion error is somewhat higher than t

In [67]:
df_excel = pd.read_excel('RS_ALL_IN_ONE_metadata_TEST.xlsx',sheet_name="Sheet1")

for index, row in df_excel.iterrows():
    print(row['key_for_all_RS'])
    print(row['rs_comment'])
    print('\n\n\n')

RS_001_MLRC_2022_01
Results — Due to many missing implementation details, we were not able to reproduce all
of the original results. Some claims can be supported by our results, but most results are
very vague. Even though the new method outperforms the baselines in certain scenarios,
we find that the superiority of the method is not as strong as presented in the original
paper.





RS_002_MLRC_2022_02
Results — We have reproduced the experiments done on two selected tasks, and com‐
pared their results with the reported results. Although our experimental results are not
identical to the reported ones, we can validate the claims made by the original study
according to these results.





RS_003_MLRC_2022_03
Results — We verify the authors’ claims by replicating the experiments presented in [1].
All of our experiments show identical results to the ones presented in [1], apart from per‐
turbation testing for which we provide an additional in depth analysis. We also provide
an analysis of